In [53]:
from pymongo import MongoClient
import pymongo
import datetime
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm

In [54]:
client = MongoClient(host='114.70.235.37', port=27017)
print(client.list_database_names())
db = client['mydb'].xy

['admin', 'config', 'local', 'mydb']


In [64]:
def make_input():
    result= db.find().sort([("date", pymongo.DESCENDING)]).limit(2)

    data = list(result)[0]
    start_date = data["date"] - datetime.timedelta(seconds=5)
    seq = list(db.find({"date": {'$gte' : start_date, '$lt' : data["date"]}}))
    
    
    list_data = list()
    
    for j in seq:
        del j['date']
        del j['_id']

        list_data.append(list(j.values()))
        
    if(len(list_data) < 30):
        return
            
    index = list(range(5, len(list_data)-1, int(len(list_data)/30)))
    list_data = np.array(list_data, dtype=np.float32)
    
    y_false = np.array([0])
    list_data = list_data[index]
    list_data = np.array([list_data])
    
    tmp_data = TensorDataset(torch.from_numpy(data_x), torch.from_numpy(data_y))
    tmp_loader = DataLoader(tmp_data, shuffle=False, batch_size=batch_size)
    
    return tmp_loader

In [56]:
# dataloaders
batch_size = 32
device='cuda'

class SentimentLSTM(nn.Module):

    def __init__(self, seq_size, output_size, hidden_dim, n_layers):
        
        super(SentimentLSTM, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        # embedding and LSTM layers
        self.lstm = nn.LSTM(seq_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        
        # linear and sigmoid layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        lstm_out, hidden = self.lstm(x)
        ## lstm_out.shape [batch, seq_len, hidden_dim]
        ## hidden [batch, hidden_dim]

        # fully connected layer        
        out = self.fc(lstm_out[:, -1, :])
        ## out.shape: [n_layer * n_direction, batch, output_size]

        # sigmoid function
        sig_out = self.sig(out)
        
        # return last sigmoid output and hidden state
        return sig_out, out

In [57]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        # nn.init.uniform_(param.data, -0.1, 0.1)

In [58]:
hidden_dim = 512
n_layers = 4
lr = 0.01
output_size = 1

model = SentimentLSTM(20, output_size, hidden_dim, n_layers)
model.to('cuda')

SentimentLSTM(
  (lstm): LSTM(20, 512, num_layers=4, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)

In [59]:
model.load_state_dict(torch.load("model_weight.pth"))

<All keys matched successfully>

In [60]:
def predict(model, test_loader, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        
        accuracy = (pred==labels).sum().item() / 32
        acc_list.append(accuracy)

        #print('Prediction value, pre-rounding: ', output.item())

        # print custom response based on whether test_review is pos/neg
    print(acc_list)
    print(np.mean(acc_list))

In [71]:
def model_run(model, test_loader, sequence_length=30):
    
    model.eval()
    
    batch_size = 32
    acc_list = list()
    
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        
        output, h = model(inputs)
        
        pred = torch.round(output.squeeze())
        

        if int(pred) != 0:
            print("넘어짐 발생")

In [72]:
def run(model):
    while True:
        data_loader = make_input()
        
        if data_loader is None:
            continue
        
        model_run(model, data_loader)

In [ ]:
run(model)